<a href="https://colab.research.google.com/github/Basel-byte/Network-Anomaly-Detection/blob/main/Pr_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Lab: Clustering  

In this exercise, we will know how K-Means and Normalized Cut algorithms can be used for network anomaly detection.

# Outline
- [ 1 - Extracting Dataset](#1)
- [ 2 - Packages ](#2)
- [ 3 - Reading data and Preprocessing](#3)
  - [3.1 Reading Columns' Names](#3.1)
  - [3.2 Reading dataset](#3.2)
  - [3.3 Removing class column](#3.3)
  - [3.4 Changing Categorical features to numericals](#3.4)
  
   
- [ 4 - K-means](#4)
- [ 5 - Spectral Clustering](#5)
- [ 6 - New Clustering Algorithm](#6)
- [ 7 - Clustering Evaluation](#7)

  

<a name="1"></a>
## 1 - Extracting Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# do not run if you have unzipped the dataset
!gzip -d /content/drive/MyDrive/kddcup.data.gz

gzip: /content/drive/MyDrive/kddcup.data.gz: No such file or directory


<a name="2"></a>
## 2 - Packages 

First, let's run the cell below to import all the packages that you will need during this assignment.
- [numpy](https://numpy.org/) is the fundamental package for scientific computing with Python.
<!-- - [matplotlib](http://matplotlib.org) is a popular library to plot graphs in Python. -->
<!-- - [tensorflow](https://www.tensorflow.org/) a popular platform for machine learning. -->
- [pandas](https://pandas.pydata.org/) is open source data analysis and manipulation tool.

In [2]:
import numpy as np
import pandas as pd
import os
import requests
from enum import Enum
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from scipy.linalg import eigh

<a name="3"></a>
## 3 - Reading data and Preprocessing

<a name="3.1"></a>
#### Reading columns' names 

In [3]:
DATASET_COLUMNS_FILE = "/content/drive/MyDrive/kddcup1999_columns.txt"
column_types =[]

with open(DATASET_COLUMNS_FILE, 'r') as file:
    column_labels: str = file.read()

column_regex: re.Pattern = re.compile(r"^(?P<column_name>\w+): (?P<data_type>\w+)\.$")
for column_type in column_labels.splitlines()[:]:
    match = column_regex.match(column_type)
    column_types.append(match.group("column_name"))
del column_labels

<a name="3.2"></a>
#### reading the dataset

In [4]:
data = pd.read_csv("/content/drive/MyDrive/kddcup.data", header=None)
data.columns = column_types

<a name="3.3"></a>
#### Removing class column

In [5]:
data_without_labels = data.drop(columns=["class"])
labels = data["class"]
del data
del column_types

<a name="3.4"></a>
### Change the categorical features to numericals

In [6]:
def convert_string_to_numeric(data_frame):
  # df_copy = data_frame.copy()
  for col in data_frame:
    if data_frame[col].dtypes == object:
      my_dict = {elem: index for index, elem in enumerate(data_frame[col].unique())}
      data_frame[col].replace(my_dict, inplace=True)
  return data_frame

In [7]:
def convert_string_to_binary_numeric(data_frame):
  # df_copy = data_frame.copy()
  for col in data_frame:
    if data_frame[col].dtypes == object:
      df1 = pd.get_dummies(data_frame[col])
      data_frame = pd.concat([data_frame, df1], axis=1).reindex(data_frame.index)
      data_frame.drop(col, axis=1, inplace=True)
  return data_frame

In [8]:
# use either of them but not both
# convert_string_to_numeric(data_without_labels.copy())

data_without_labels = convert_string_to_binary_numeric(data_without_labels).values.astype(np.float32)

<a name="3"></a>
## 3 - K-means



In [9]:
def Kmeans(data_frame_no_label, k = 3):
  print("fol y dawly")
  return  

<a name="4"></a>
## 4 - Spectral clustering


### 4.1 Getting Similarity Matrix

In [12]:
def get_laplacian_degree(data_no_label, gamma=0.000000000005):
  sim_matrix = rbf_kernel(data_no_label, gamma=gamma)
  del data_no_label
  np.fill_diagonal(sim_matrix, 0)
  deg_matrix = np.sum(sim_matrix, axis=1)
  np.fill_diagonal(sim_matrix, deg_matrix)
  print(deg_matrix[deg_matrix == 0].shape)
  return sim_matrix, np.diag(deg_matrix)

In [13]:
def spectral_clustering(data_no_label, k = 3, is_train = True):
  if is_train:
    data_no_label, dummy = train_test_split(data_no_label, train_size=0.0025, random_state=42, stratify=labels)
  del dummy
  laplacian_mat, degree_mat = get_laplacian_degree(data_no_label) # deg_mat is a vector
  eigenvalues, eigenvectors = eigh(laplacian_mat, degree_mat)
  del eigenvalues
  eigenvectors = eigenvectors[:, :k] / np.linalg.norm(eigenvectors[:, :k], ord=2, axis=1, keepdims=True)
  return Kmeans(eigenvectors, k)
spectral_clustering(data_without_labels, 3, True)

(0,)
fol y dawly


<a name="5"></a> 
## 5-  New Clustering Algorithm

In [ ]:
# def new_clustering_algorithm(data_frame_no_label, k = 3):
  

<a name="6"></a>
## 6- Clustering Evaluation